# 딥러닝모델과추론 과제 4

- 딥러닝모델과 추론
- 데이터과학융합스쿨 20173237 심창우

In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

In [2]:
# 데이터 불러오기
dat = pd.read_csv("test.csv")

# 키 - 몸무게 데이터 2차원 배열로 변환
height = np.array(dat["Height"])
height = height.reshape(len(height), 1)
weight = np.array(dat["Weight"])
weight = weight.reshape(len(weight), 1)

# 수축기 - 이완기 데이터 2차원 배열로 변환
SBP = np.array(dat["SBP"])
SBP = SBP.reshape(len(SBP), 1)
DBP = np.array(dat["DBP"])
DBP = DBP.reshape(len(DBP), 1)

# 혈당수치 - 당뇨여부 데이터 2차원 배열로 변환

# String을 지원하지 않기 때문에 당뇨병 유무(Y, N)를 숫자로 변환
def transform_YesNo(dm):
  if dm == 'Y':
    return 1
  else:
    return 0

Sugar = np.array(dat["Sugar"])
Sugar = Sugar.reshape(len(Sugar), 1)

# 당뇨병 유무(Y, N) 변환(1, 0 함수 호출 후 array 생성
DM = np.array([transform_YesNo(i) for i in dat["DM"]])
DM = DM.reshape(len(DM), 1)

# 키 - 몸무게 정규화
mm_scaler_weight = preprocessing.MinMaxScaler()
height = mm_scaler_weight.fit_transform(height)
weight = mm_scaler_weight.transform(weight)

# 수축기 - 이완기 정규화
mm_scaler_SBP = preprocessing.MinMaxScaler()
SBP = mm_scaler_SBP.fit_transform(SBP)
DBP = mm_scaler_SBP.transform(DBP)

# 혈당수치 - 당뇨여부 정규화. 당뇨병 유무는 1, 0값이기 때문에 하지 않음
mm_scaler_Sugar = preprocessing.MinMaxScaler()
Sugar = mm_scaler_Sugar.fit_transform(Sugar)

In [3]:
# 키 - 몸무게 모델
model_height = Sequential()
model_height.add(Dense(1, input_dim=1))

model_height.summary()

# 수축기 - 이완기 모델
model_SBP = Sequential()
model_SBP.add(Dense(1, input_dim=1))

model_SBP.summary()

# 혈당수치 - 당뇨여부 모델
model_Sugar = Sequential()

# relu -> 0 또는 1이기 때문에 activation sigmoid 사용
model_Sugar.add(Dense(1, input_dim=1, activation='relu'))
model_Sugar.add(Dense(1, activation='sigmoid'))

model_Sugar.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 2         
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 1)                 2         
_________________________________________________________________
dense_3 (Dense)         

- lost(cost) function = mse
- optimizer = Stochastic Gradient Descent

In [4]:
# 키 - 몸무게 모델 컴파일
model_height.compile(loss='mse', optimizer='sgd') 

# 수축기 - 이완기 모델 컴파일
model_SBP.compile(loss='mse', optimizer='sgd') 

# 혈당수치 - 당뇨여부 모델 컴파일. 결과값이 0 또는 1 즉 이진분류이기 때문에 binary_crossentropy 사용함
model_Sugar.compile(loss='binary_crossentropy', optimizer='sgd') 

In [5]:
# 키 - 몸무게 모델 적합 (model fitting)
fit = model_height.fit(height, weight, epochs=5000, batch_size=20, verbose=False)

# 수축기 - 이완기 모델 적합 (model fitting)
fit = model_SBP.fit(SBP, DBP, epochs=5000, batch_size=20, verbose=False)

# 혈당수치 - 당뇨여부 모델 적합 (model fitting)
fit = model_Sugar.fit(Sugar, DM, epochs=15000, batch_size=20, verbose=False)

In [8]:
# 과제 4-1, 4-2 , 4-3 예측값

# 키 입력
input_height = 170

input_mm_height = mm_scaler_weight.transform(np.array([input_height]).reshape(-1, 1))

weight_predict = model_height.predict(input_mm_height)
weight_predict = mm_scaler_weight.inverse_transform(weight_predict)
print("1번)", "키가", input_height, "cm 일 때 예측 몸무게는 약", round(weight_predict[0][0]), "kg")

# 수축기 입력 
input_SBP = 140

input_mm_SBP = mm_scaler_SBP.transform(np.array([input_SBP]).reshape(-1, 1))

DBP_predict = model_SBP.predict(input_mm_SBP)
DBP_predict = mm_scaler_SBP.inverse_transform(DBP_predict)
print("2번)", "수축기가", input_SBP, "mmHG 일 때 예측 이완기 혈압은 약", round(DBP_predict[0][0]), "mmHG")

# 혈당 입력 
input_Sugar = 90

input_mm_Sugar = mm_scaler_Sugar.transform(np.array([input_Sugar]).reshape(-1, 1))
DM_predict = model_Sugar.predict(input_mm_Sugar)
print("3번)", "혈당수치가", input_Sugar, "일 때 당뇨에 걸릴 확률은", round(DM_predict[0][0]*100), "% 이다")

1번) 키가 170 cm 일 때 예측 몸무게는 약 69 kg
2번) 수축기가 140 mmHG 일 때 예측 이완기 혈압은 약 93 mmHG
3번) 혈당수치가 90 일 때 당뇨에 걸릴 확률은 0 % 이다
